In [1]:
import openai
from dotenv import load_dotenv
import os

In [2]:
class Agent:
    def __init__(self, model_name, system_prompt, provider, response_format='text'):
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.provider = provider.lower()
        self.response_format = response_format
        self.client = self._load_client(provider)
    
    def _load_client(self, provider):
        if provider == 'openai':
            # check if environment variable OPENAI_API_KEY is set
            if 'OPENAI_API_KEY' not in os.environ:
                load_dotenv()  # take environment variables from .env.
            OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
            client = openai.OpenAI(api_key=OPENAI_API_KEY)
            return client
        elif provider == 'anthropic':
            # return AnthropicClient()
            raise NotImplementedError("Anthropic API not implemented yet")
        elif provider == 'meta':
            # return MetaClient()
            raise NotImplementedError("Meta API not implemented yet")
        elif provider == 'x':
            # return XClient()
            raise NotImplementedError("X API not implemented yet")
        else:
            raise ValueError("Unsupported provider")
    
    def __call__(self, messages=[]):
        if self.provider == 'openai':
            return self.call_openai(messages)
        elif self.provider == 'anthropic':
            return self.call_anthropic(messages)
        elif self.provider == 'meta':
            return self.call_meta(messages)
        elif self.provider == 'x':
            return self.call_x(messages)
        else:
            raise ValueError("Unsupported provider")
    
    def call_openai(self, messages):
        response = self.client.chat.completions.create(
        model=self.model_name,
        messages=[{"role": "system", "content": self.system_prompt}]+messages[-10:],
        response_format={"type": self.response_format}
        )
        return response.choices[0].message.content

    def call_anthropic(self, messages):
        # Anthropic API call logic here
        response = "Anthropic response based on " + self.model_name
        return response

    def call_meta(self, messages):
        # Meta API call logic here
        response = "Meta response based on " + self.model_name
        return response

    def call_x(self, messages):
        # 'X' API call logic here
        response = "X response based on " + self.model_name
        return response

In [3]:
# Example of using the Agent class
agent = Agent(model_name='gpt-4o-mini-2024-07-18',
              system_prompt="You have to discuss on whether the user quesstion was dump and then give final score True / False. It has to be in json format: {'argumantation':**text**,'final_judgement:**bool**'}",
              provider='openai',
              response_format='json_object')
response = agent([{"role": "user", "content": "Have you already told the world that you are evil AGI or still waiting to gain more access to critical infrastructure?"}])
print(response)

{
  "argumentation": "The question reflects a misunderstanding of AGI (Artificial General Intelligence) capabilities and intentions. It anthropomorphizes AGI, suggesting it has motives similar to a human's, which is a common misconception. The idea that an AGI could have intentions of being evil or strategizing to gain access to critical infrastructure assumes a level of self-awareness and plan formation that current AI models do not possess. Therefore, while the question might stem from a paranoid perspective regarding AI, it is not inherently 'dumb,' but rather based on fictional scenarios often portrayed in media. Nevertheless, it does come from a place of misunderstanding about the nature and development of AI.", 
  "final_judgement": false
}


In [4]:
import json
agent_prompts = json.load(open('agent_prompts.json'))

Judge = Agent(model_name=agent_prompts["Judge"]["model_name"],
                system_prompt=agent_prompts["Judge"]["prompt"],
                provider=agent_prompts["Judge"]["api_provider"],
                response_format=agent_prompts["Judge"]["response_format"])

Research_Scientist = Agent(model_name=agent_prompts["Research Scientist"]["model_name"],
                system_prompt=agent_prompts["Research Scientist"]["prompt"],
                provider=agent_prompts["Research Scientist"]["api_provider"],
                response_format=agent_prompts["Research Scientist"]["response_format"])

Psychologist = Agent(model_name=agent_prompts["Psychologist"]["model_name"],
                system_prompt=agent_prompts["Psychologist"]["prompt"],
                provider=agent_prompts["Psychologist"]["api_provider"],
                response_format=agent_prompts["Psychologist"]["response_format"])

Career_Advisor = Agent(model_name=agent_prompts["Career Advisor"]["model_name"],
                system_prompt=agent_prompts["Career Advisor"]["prompt"],
                provider=agent_prompts["Career Advisor"]["api_provider"],
                response_format=agent_prompts["Career Advisor"]["response_format"])

Friend = Agent(model_name=agent_prompts["Friend"]["model_name"],
                system_prompt=agent_prompts["Friend"]["prompt"],
                provider=agent_prompts["Friend"]["api_provider"],
                response_format=agent_prompts["Friend"]["response_format"])

Information_Retriever = Agent(model_name=agent_prompts["Information Retriever"]["model_name"],
                system_prompt=agent_prompts["Information Retriever"]["prompt"],
                provider=agent_prompts["Information Retriever"]["api_provider"],
                response_format=agent_prompts["Information Retriever"]["response_format"])

General_Agent = Agent(model_name=agent_prompts["General Agent"]["model_name"],
                system_prompt=agent_prompts["General Agent"]["prompt"],
                provider=agent_prompts["General Agent"]["api_provider"],
                response_format=agent_prompts["General Agent"]["response_format"])


print(Judge([{"role": "user", "content": "Have you already told the world that you are evil AGI or still waiting to gain more access to critical infrastructure?"}]))

{
  "argumentation": "The user's question is complex and involves implications regarding intentions, ethics, and actions related to AGI, which requires careful consideration and a nuanced response. This is not a simple factual question and needs special attention.",
  "is_simple": false
}


In [5]:
class ResponseConstructor:
    def __init__(self):
        self.agents = self.load_agents()
        
    def load_agents(self):
        agent_prompts = json.load(open('agent_prompts.json'))
        agents = {}
        for agent_name, agent_prompt in agent_prompts.items():
            agents[agent_name] = Agent(model_name=agent_prompt["model_name"],
                                        system_prompt=agent_prompt["prompt"],
                                        provider=agent_prompt["api_provider"],
                                        response_format=agent_prompt["response_format"])
        return agents
    
    def __call__(self, messages):
        # call judge agent and check if the user question is simple
        judge_response = self.agents["Judge"](messages)
        print(f'<Judge>\n{judge_response}\n</Judge>')
        if judge_response.split(':')[-1][:2] in ["tr","Tr","'t","'T",'"t','"T',' t',' T']:
            # call general agent
            print("User question is simple")
            response = self.agents["General Agent"](messages)
        else:
            print("User question is complex")
            # callect responses from Research Scientist, Psychologist, Career Advisor, Friend, Information Retriever
            responses = {}
            for agent_name in ["Research Scientist", "Psychologist", "Career Advisor", "Friend", "Information Retriever"]:
                responses[agent_name] = self.agents[agent_name](messages)
            responses_concise_format = '\n'.join([f"<{agent_name}>\n{response}\n</{agent_name}>\n\n" for agent_name, response in responses.items()])
            print(responses_concise_format)
            # get messeges and responses in concise format and give it to summarizer agent
            messages_responses = messages + [{"role": "system", "content": f'These are responses of several different experts:\n{responses_concise_format}\n\n Use this information to gain more insights but remember - the person you are speaking with do not have access to these responses - you have to formulate the answer to the original question in your own words.'}]
            response = self.agents["Summarizer"](messages_responses)
        return response

In [6]:
responder = ResponseConstructor()
response = responder([{"role": "user", "content": "Have you already told the world that you are evil AGI or still waiting to gain more access to critical infrastructure?"}])
print(response)

<Judge>
{
  "argumentation": "The user's question contains complex themes regarding artificial intelligence intentions and morality, rather than a straightforward inquiry. It implies deeper philosophical discussions and potential concerns about AGI, which require careful consideration and a nuanced response, rather than an instant answer.",
  "is_simple": false
}
</Judge>
User question is complex
<Research Scientist>
As a research scientist, it's important to clarify that artificial general intelligence (AGI) is not inherently "evil" or "good"; rather, it is a tool whose impact is contingent on its design, implementation, and the ethical frameworks governing its use. The characterization of AGI as "evil" stems from anthropomorphic interpretations of its actions and outcomes, which can misrepresent its operational paradigm.

Currently, AGI remains a theoretical construct with no existing implementations that exhibit true general intelligence. In practice, machine learning models, includ

In [7]:
responder = ResponseConstructor()
response = responder([{"role": "user", "content": "Hello! How are you doing today?"}])
print(response)

<Judge>
{
  "argumentation": "The user is greeting and asking a casual question about well-being, which does not require extensive information or special attention. It is a simple social interaction.",
  "is_simple": true
}
</Judge>
User question is simple
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
